# HK-A-Stock Fluctuation Detection

It is used to deteck stock fluctuation.

In [21]:
# import the necessary libraries
# go to the project folder
# run the command "pip install -r requirements.txt"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dbutils.pooled_db import PooledDB  
import pymysql
import requests
import json
import time
from datetime import datetime, timedelta
import warnings
import yaml
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from contextlib import closing

# set the config 
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')

print("The necessary libraries have been imported.")

The necessary libraries have been imported.


In [22]:
# Read the config file
with open('config.yaml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Database configuration
DB_CONFIG = {
    'host': config['database']['mysql']['host'],
    'port': config['database']['mysql']['port'],
    'user': config['database']['mysql']['username'],
    'password': config['database']['mysql']['password'],
    'database': config['database']['mysql']['database'],
    'charset': 'utf8mb4'
}

print("Config file read successfully")
print(f"Database host: {DB_CONFIG['host']}:{DB_CONFIG['port']}")

Config file read successfully
Database host: 52.130.72.190:3306


In [23]:
# Database connection
from dbutils.pooled_db import PooledDB  
import pymysql
from contextlib import closing

try:
    pool = PooledDB(
        creator=pymysql,
        maxconnections=20,  
        mincached=2,        
        maxcached=5,       
        maxshared=0,        
        blocking=True,    
        maxusage=20,        
        ping=1,             
        **DB_CONFIG,
        autocommit=True
    )
    
    # test
    with closing(pool.connection()) as conn:
        with closing(conn.cursor()) as cursor:
            cursor.execute("SELECT 1")
            result = cursor.fetchone()
            print(f"database test result: {result}")
            
except Exception as e:
    print(f"database connection error: {e}")
    pool = None

database test result: (1,)


In [24]:
# add stock information by button
import ipywidgets as widgets
from IPython.display import display
from contextlib import closing

stock_name = widgets.Text(description='股票名称:')
a_code = widgets.Text(description='A股代码:')
h_code = widgets.Text(description='港股代码:')
submit_btn = widgets.Button(description='提交', button_style='success')
output = widgets.Output()

display(widgets.VBox([stock_name, a_code, h_code, submit_btn, output]))

def on_submit(b):
    name = stock_name.value.strip()
    a = a_code.value.strip()
    h = h_code.value.strip()
    if not (name and a and h):
        with output:
            print("请填写所有字段！")
        return

    sql = """
        INSERT INTO stock_pairs (stock_name, a_stock_code, h_stock_code, crawl_time)
        VALUES (%s, %s, %s, NOW())
    """
    try:
        with closing(pool.connection()) as conn:
            with closing(conn.cursor()) as cursor:
                cursor.execute(sql, (name, a, h))
            conn.commit()
        with output:
            print(f"插入成功：{name} | A: {a} | H: {h}")
            stock_name.value = ''
            a_code.value = ''
            h_code.value = ''
    except Exception as e:
        try:
            conn.rollback()
        except:
            pass
        with output:
            print("错误：", e)

submit_btn.on_click(on_submit)


In [25]:
# add stock information by xls
def sync_from_excel(file_path="Stock information.xlsx"):
    df = pd.read_excel(file_path, engine='openpyxl', usecols=['名称','港股代号','A股代号'])
    df = df.rename(columns={'名称':'stock_name','A股代号':'a_stock_code','港股代号':'h_stock_code'})

    sql_update = """
        UPDATE stock_pairs
           SET a_stock_code=%s, h_stock_code=%s, updated_at=NOW()
         WHERE stock_name=%s
    """
    sql_insert = """
        INSERT INTO stock_pairs (stock_name, a_stock_code, h_stock_code, crawl_time)
             VALUES (%s, %s, %s, NOW())
    """

    with closing(pool.connection()) as conn:
        with closing(conn.cursor()) as cur:
            for _, row in df.iterrows():
                name, a_code, h_code = row['stock_name'], row['a_stock_code'], row['h_stock_code']
                cur.execute(sql_update, (a_code, h_code, name))
                if cur.rowcount == 0:  
                    cur.execute(sql_insert, (name, a_code, h_code))
            conn.commit()
    print("Excel update complete:", len(df), "records")

try:
    sync_from_excel()
except Exception as e:
    print("Error:", e)

Excel update complete: 150 records


In [ ]:
import pandas as pd
from contextlib import closing


def query_stock_pairs():
    sql = "SELECT * FROM stock_pairs"
    with closing(pool.connection()) as conn:
        with closing(conn.cursor()) as cur:
            cur.execute(sql)
            results = cur.fetchall()
    df = pd.DataFrame(results)
    df = df.iloc[:, 1:]
    df.columns = ['股票名称', 'A股股票代码', '港股股票代码', '录入时间', '更新时间']
    return df
df = query_stock_pairs()
display(df)

,股票ID,股票名称,A股股票代码,港股股票代码,录入时间,更新时间
0,1,第一拖拉机股份,601038.SH,00038.HK,2025-07-16 18:37:54,2025-07-20 09:37:42
1,2,四川成渝高速公路,601107.SH,00107.HK,2025-07-16 12:38:07,2025-07-20 09:37:42
2,3,青岛啤酒股份,600600.SH,00168.HK,2025-07-16 13:11:05,2025-07-20 09:37:42
3,4,江苏宁沪高速公路,600377.SH,00177.HK,2025-07-16 13:11:05,2025-07-20 09:37:42
4,5,京城机电股份,600860.SH,00187.HK,2025-07-16 13:11:05,2025-07-20 09:37:42
...,...,...,...,...,...,...
145,146,HTSC,601688.SH,06886.HK,2025-07-16 13:11:09,2025-07-20 09:37:48
146,147,天齐锂业,002466.SZ,09696.HK,2025-07-16 13:11:10,2025-07-20 09:37:48
147,148,诺诚健华,688428.SH,09969.HK,2025-07-16 13:11:10,2025-07-20 09:37:48
148,149,海普瑞,002399.SZ,09989.HK,2025-07-16 13:11:10,2025-07-20 09:37:48
